In [1]:
import os
import sys
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Layer, Dense
from tensorflow.keras.optimizers import Adam
import tensorflow_probability as tfp
from vocab import Vocabulary
from env import ConceptData
from create_data import addFile
import tensorflow as tf
import random
import numpy as np
import csv
import re
import os
import create_data
from xml.dom import minidom
import xml.etree.ElementTree as ET
from vocab import Vocabulary
from env2 import ConceptData
from create_data import addFile
import agents
from agents import find_lengths
import sender2
import game

In [2]:
sender_encoder = sender2.Sender(num_options=10, batch_size=32)
sender_only_target = sender2.SenderOnlyTarget(batch_size=32)
sender = sender2.Sender_LSTM(agent = sender_only_target, vocab_size=99, embed_dim=44, num_cells=1, hidden_size=1, max_len=10, see_all_input=False)
receiver_decoder = agents.Receiver(num_options=10)
receiver_encoder = agents.Receiver_LSTM(agent=receiver_decoder, vocab_size=99, embed_dim=44, hidden_size=44, masking_value=99)

In [3]:
vocabulary = Vocabulary()

def addFile(name):
    file_name = name + "_structured_final.xml"
    file = minidom.parse(os.path.join(os.path.join('visa_dataset', 'UK'), file_name))
    concepts = file.getElementsByTagName('concept')

    for concept in concepts:
        vocabulary.addConcept(concept)


addFile("ANIMALS")
addFile("APPLIANCES")
addFile("ARTEFACTS")
addFile("CLOTHING")
addFile("CONTAINER")
addFile("DEVICE")
addFile("FOOD")
addFile("HOME")
addFile("INSTRUMENTS")
addFile("MATERIAL")
addFile("PLANTS")
addFile("STRUCTURES")
addFile("TOOLS")
addFile("TOYS")
addFile("VEHICLES")
addFile("WEAPONS")

for concept in vocabulary.concept_list:
    vocabulary.parseConcept(concept)
    
data = ConceptData(vocabulary, 9, 32)

In [4]:
input_concepts, sender_input, targets, receiver_input = data.getInputDifferent()
#print(sender_input.shape)

In [5]:
#sender_encoded = sender_encoder(sender_input)
#sender_input = tf.squeeze(tf.convert_to_tensor(sender_input))
#print(sender_input)
sender_input = tf.transpose(sender_input, [1,0,2])
sender_input = sender_input.numpy()

message, log_prob_s, entropy_s, state_h = sender(sender_input)
print("message:")
print(message)
print("logits:")
print(log_prob_s)
print("entropy:")
print(entropy_s)
print("state_h:")
print(state_h)

message:
tf.Tensor(
[[86 87  2 36  2 46 96 16 80  0  0  0  0  0  0  0  0  0]
 [22 55 83 61 39 36 30 82 74  0  0  0  0  0  0  0  0  0]
 [37 35  7 57 93 26 24 72 25  0  0  0  0  0  0  0  0  0]
 [79 13 37 94 58  8 83 57 33  0  0  0  0  0  0  0  0  0]
 [53 76  0  4 70 27 14 41 34  0  0  0  0  0  0  0  0  0]
 [33 46 23  7 85 34 85 26 32  0  0  0  0  0  0  0  0  0]
 [87 29 96 29  1 86 87 15 91  0  0  0  0  0  0  0  0  0]
 [52 44 41 26 14 29  5 42 59  0  0  0  0  0  0  0  0  0]
 [80 13 71 11 72 57  3 79 50  0  0  0  0  0  0  0  0  0]
 [15 57 41 32 77 69 29 94 25  0  0  0  0  0  0  0  0  0]
 [18 92 29  8 87  1 74 44  6  0  0  0  0  0  0  0  0  0]
 [75  6 75  4 62 82 50 97 47  0  0  0  0  0  0  0  0  0]
 [26  2 59 73 65 68 98 35 54  0  0  0  0  0  0  0  0  0]
 [59  6 56 70 23 65 35 61 29  0  0  0  0  0  0  0  0  0]
 [ 5 63 58 36 28 66 15 96 10  0  0  0  0  0  0  0  0  0]
 [70  2 43 66 23 16 97 76  0  0  0  0  0  0  0  0  0  0]
 [ 3 78 24 47 15  3 51 10 54  0  0  0  0  0  0  0  0  0]
 [92 27 97 

In [6]:

#einer- Dimension rausbekommen
receiver_input = tf.squeeze(tf.convert_to_tensor(receiver_input))
#richtige Reihenfolge der Dimensionen
receiver_input2 = tf.transpose(receiver_input, [1,0,2])
receiver_input2 = receiver_input2.numpy()

sample, log_prob_r, entropy_r, last_hidden = receiver_encoder(message=message, batch_size=32, max_len=10, receiver_input=receiver_input2)

print('entropy_r_shape ', entropy_r.shape)
print('entropy_s_shape ', entropy_s.shape)
message_lengths = find_lengths(message)
print('message_lengths: ', message_lengths)

effective_entropy_s = tf.zeros_like(entropy_r)
effective_entropy_s = np.array(effective_entropy_s)
effective_log_prob_s = tf.zeros_like(log_prob_r)
effective_log_prob_s = np.array(effective_log_prob_s)

#print(message.shape)

for k in range(message.shape[0]):
    for l in range(message.shape[1]):
        not_eosed = float(k < message_lengths[k])
        effective_entropy_s[k] += entropy_s[k, l] * not_eosed
        effective_log_prob_s[k] += log_prob_s[k, l] * not_eosed
        
effective_entropy_s = tf.convert_to_tensor(effective_entropy_s)
effective_log_prob_s = tf.convert_to_tensor(effective_log_prob_s)
        
receiver_output = []
for i in range(32):
    receiver_output.append(input_concepts[i][sample[i]])
    #print(targets[i] == receiver_output[i])

#targets = tf.constant(targets)
#receiver_output = tf.constant(receiver_output)

acc = np.sum(targets == receiver_output) / targets.shape[0]
acc

entropy_r_shape  (32,)
entropy_s_shape  (32, 18)
message_lengths:  [10, 10, 10, 10, 3, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 9, 10, 6, 3, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10]


0.09375

In [ ]:
def loss(input_concepts, receiver_output, targets):
    """
    receiver_output ist was von receiver_sampling zurückgegeben wird
    LABELS PRINTEN IN OSSSENKOPF NOTEBOOK
    """
    guesses = []
    
    for i in range(len(receiver_output)):
        guesses.append(input_concepts[i][receiver_output[i]])
            
    guesses = tf.convert_to_tensor(guesses)
    targets = tf.convert_to_tensor(targets)
    acc = np.sum(guesses == targets) / guesses.shape[0]
    
    return -acc

In [ ]:
sender_entropy_coeff = 0.015 #wie bei Ossenkopf
receiver_entropy_coeff = 0.0 # wie bei Ossenkopf


game = game = game.Game(sender=sender,
                receiver=receiver_encoder,
                main_loss=loss,
                sender_entr_coeff=sender_entropy_coeff,
                receiver_entr_coeff=receiver_entropy_coeff,
                batch_size=32,
                max_len=10)

In [ ]:
optim = tf.keras.optimizers.Adam(learning_rate = 1e-5)

for _ in range(10):
    with tf.GradientTape() as tape:
    
        loss, prev_hidden, last_hidden = game(input_concepts, sender_input, targets, receiver_input)
        game_gradients = tape.gradient(loss, game.trainable_variables)
    
    optim.apply_gradients(zip(game_gradients, game.trainable_variables))
